In [16]:
import json
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os

# 0. Loading Dataset Contating Korean Lyrics

In [3]:
json_final = open("/Users/jeongseohyeong/Dropbox/Yonsei/Fall_18/Data Science/project/seohyeong_lyric.json", encoding='utf-8').read()
data_final = json.loads(json_final)
df = pd.read_json(json_final)

In [4]:
df.head()

,id,lyric,title
0,5398990,시계가 반대로 돌아가고 있어TV속 영화가 되감아지고 있어내렸던 빗물이 올라가고 있어...,이 소설의 끝을 다시 써보려 해
1,9634954,La La La La La La La La La La La LaLa La La La...,러시안 룰렛 (Russian Roulette)
2,9614761,그림 같은 집이 뭐 별거겠어요 어느 곳이든 그대가 있다면 그게 그림이죠빛나는 하루가...,그대라는 사치
3,9647324,Hey hey hey heyHey hey hey heyHey hey hey hey뜨...,Dancing King
4,9613173,평범한 목요일 밤널 데려 갈게 어디든일주일 중에 네가제일 지쳐 있을 오늘Wanna ...,목요일 밤 (Feat. 빈지노)


In [8]:
# 역주행 변수: label이란 이름으로 생성 (if 역주행곡: 1, if not: 0)
target = ["3111583", "3607859", "3701333", "5473905", "4163116", "3352959", "3172166", "3349210", "3319849", "4148967", "4016085", "2793050", "3666573", "3753303", "4840271", "4099280", "3689468", "3110565", "5704397", "5713422", "5385974", "5804040", "5820368", "7891119", "5398990", "9620469", "30132687", "30367365", "30414090", "30467550", "30486509", "30383758", "30086173", "30755375", "30816860", "30699142", "31052421", "31151836", "30962526", "31314144"]

df['id']= df['id'].astype(str)
df['label'] = np.where(df['id'].isin(target), 1, 0)

# save only non-duplicated
df_small = df.drop_duplicates()

# 1. Tokenization Using Twitter

In [17]:
twitter = Twitter()
def tokenizer_twitter_morphs(doc):
    return twitter.morphs(doc)

def tokenizer_twitter_noun(doc):
    return twitter.nouns(doc)

def tokenizer_twitter_pos(doc):
    return twitter.pos(doc, norm=True, stem=True)

In [19]:
df_small['token_lyric_twitter'] = df_small['lyric'].apply(tokenizer_twitter_morphs)

/Users/jeongseohyeong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# 2. Importing Korean NRC Sentiment Emotion Lexicon

In [47]:
xlsx = pd.read_excel("/Users/jeongseohyeong/Dropbox/Yonsei/Fall_18/Data Science/project/NRC-Sentiment-Emotion-Lexicons/NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/KO_NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx")
xlsx.sample(15)

,English (en),Korean (ko),Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
7743,materiality,유형,0,0,0,0,0,0,0,0,0,0
3229,decoy,유인 못,0,0,0,0,0,0,0,0,1,0
4479,evaluate,평가하다,0,0,0,0,0,0,0,0,0,0
10471,remake,다시 만들다,1,0,0,0,0,0,0,0,0,0
5271,fulfill,다하다,1,0,0,0,0,0,1,0,0,0
9554,posture,자세,0,0,0,0,0,0,0,0,0,0
3632,disbursement,지불,0,0,0,0,0,0,0,0,0,0
7034,keel,용골,0,0,0,0,0,0,0,0,0,0
6472,indestructible,불멸의,1,0,0,0,0,0,0,0,0,1
10042,quadrant,사분면,0,0,0,0,0,0,0,0,0,0


In [54]:
# creat a new data table 'emo_df' to count sentiments
emo_df = df_small[['id', 'title', 'label', 'token_lyric', 'token_lyric_twitter']]

emo_df['positive'] = 0.0
emo_df['negative'] = 0.0
emo_df['anger'] = 0.0
emo_df['anticipation'] = 0.0
emo_df['disgust'] = 0.0
emo_df['fear'] = 0.0
emo_df['joy'] = 0.0
emo_df['sadness'] = 0.0
emo_df['surprise'] = 0.0
emo_df['trust'] = 0.0

# make a list of Korean dictionary words
lst = [str(x) for x in xlsx['Korean (ko)']]

In [155]:
for i in range(0, len(emo_df)):
    print("++++++++++++", i, "번째 case 분석중++++++++++++++") 
    for item in emo_df.token_lyric_twitter.iloc[i]:
        if item in lst:
            temp = xlsx[xlsx['Korean (ko)'] == item].iloc[0] 
            emo_df.positive.iloc[i] = emo_df.positive.iloc[i] + temp.Positive
            emo_df.negative.iloc[i] = emo_df.negative.iloc[i] + temp.Negative
            emo_df.anger.iloc[i] = emo_df.anger.iloc[i] + temp.Anger
            emo_df.anticipation.iloc[i] = emo_df.anticipation.iloc[i] + temp.Anticipation
            emo_df.disgust.iloc[i] = emo_df.disgust.iloc[i] + temp.Disgust
            emo_df.fear.iloc[i] = emo_df.fear.iloc[i] + temp.Fear
            emo_df.joy.iloc[i] = emo_df.joy.iloc[i] + temp.Joy
            emo_df.sadness.iloc[i] = emo_df.sadness.iloc[i] + temp.Sadness
            emo_df.surprise.iloc[i] = emo_df.surprise.iloc[i] + temp.Surprise
            emo_df.trust.iloc[i] = emo_df.trust.iloc[i] + temp.Trust
        else:
            continue

            
#emo_df.sample(10) to see data table

++++++++++++ 250 번째 case 분석중++++++++++++++

/Users/jeongseohyeong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)



++++++++++++ 251 번째 case 분석중++++++++++++++
++++++++++++ 252 번째 case 분석중++++++++++++++
++++++++++++ 253 번째 case 분석중++++++++++++++
++++++++++++ 254 번째 case 분석중++++++++++++++
++++++++++++ 255 번째 case 분석중++++++++++++++
++++++++++++ 256 번째 case 분석중++++++++++++++
++++++++++++ 257 번째 case 분석중++++++++++++++
++++++++++++ 258 번째 case 분석중++++++++++++++
++++++++++++ 259 번째 case 분석중++++++++++++++
++++++++++++ 260 번째 case 분석중++++++++++++++
++++++++++++ 261 번째 case 분석중++++++++++++++
++++++++++++ 262 번째 case 분석중++++++++++++++
++++++++++++ 263 번째 case 분석중++++++++++++++
++++++++++++ 264 번째 case 분석중++++++++++++++
++++++++++++ 265 번째 case 분석중++++++++++++++
++++++++++++ 266 번째 case 분석중++++++++++++++
++++++++++++ 267 번째 case 분석중++++++++++++++
++++++++++++ 268 번째 case 분석중++++++++++++++
++++++++++++ 269 번째 case 분석중++++++++++++++
++++++++++++ 270 번째 case 분석중++++++++++++++
++++++++++++ 271 번째 case 분석중++++++++++++++
++++++++++++ 272 번째 case 분석중++++++++++++++
++++++++++++ 273 번째 case 분석중++++++++++++++
++++++++++

++++++++++++ 442 번째 case 분석중++++++++++++++
++++++++++++ 443 번째 case 분석중++++++++++++++
++++++++++++ 444 번째 case 분석중++++++++++++++
++++++++++++ 445 번째 case 분석중++++++++++++++
++++++++++++ 446 번째 case 분석중++++++++++++++
++++++++++++ 447 번째 case 분석중++++++++++++++
++++++++++++ 448 번째 case 분석중++++++++++++++
++++++++++++ 449 번째 case 분석중++++++++++++++
++++++++++++ 450 번째 case 분석중++++++++++++++
++++++++++++ 451 번째 case 분석중++++++++++++++
++++++++++++ 452 번째 case 분석중++++++++++++++
++++++++++++ 453 번째 case 분석중++++++++++++++
++++++++++++ 454 번째 case 분석중++++++++++++++
++++++++++++ 455 번째 case 분석중++++++++++++++
++++++++++++ 456 번째 case 분석중++++++++++++++
++++++++++++ 457 번째 case 분석중++++++++++++++
++++++++++++ 458 번째 case 분석중++++++++++++++
++++++++++++ 459 번째 case 분석중++++++++++++++
++++++++++++ 460 번째 case 분석중++++++++++++++
++++++++++++ 461 번째 case 분석중++++++++++++++
++++++++++++ 462 번째 case 분석중++++++++++++++
++++++++++++ 463 번째 case 분석중++++++++++++++
++++++++++++ 464 번째 case 분석중++++++++++++++
+++++++++++

++++++++++++ 633 번째 case 분석중++++++++++++++
++++++++++++ 634 번째 case 분석중++++++++++++++
++++++++++++ 635 번째 case 분석중++++++++++++++
++++++++++++ 636 번째 case 분석중++++++++++++++
++++++++++++ 637 번째 case 분석중++++++++++++++
++++++++++++ 638 번째 case 분석중++++++++++++++
++++++++++++ 639 번째 case 분석중++++++++++++++
++++++++++++ 640 번째 case 분석중++++++++++++++
++++++++++++ 641 번째 case 분석중++++++++++++++
++++++++++++ 642 번째 case 분석중++++++++++++++
++++++++++++ 643 번째 case 분석중++++++++++++++
++++++++++++ 644 번째 case 분석중++++++++++++++
++++++++++++ 645 번째 case 분석중++++++++++++++
++++++++++++ 646 번째 case 분석중++++++++++++++
++++++++++++ 647 번째 case 분석중++++++++++++++
++++++++++++ 648 번째 case 분석중++++++++++++++
++++++++++++ 649 번째 case 분석중++++++++++++++
++++++++++++ 650 번째 case 분석중++++++++++++++
++++++++++++ 651 번째 case 분석중++++++++++++++
++++++++++++ 652 번째 case 분석중++++++++++++++
++++++++++++ 653 번째 case 분석중++++++++++++++
++++++++++++ 654 번째 case 분석중++++++++++++++
++++++++++++ 655 번째 case 분석중++++++++++++++
+++++++++++

++++++++++++ 824 번째 case 분석중++++++++++++++
++++++++++++ 825 번째 case 분석중++++++++++++++
++++++++++++ 826 번째 case 분석중++++++++++++++
++++++++++++ 827 번째 case 분석중++++++++++++++
++++++++++++ 828 번째 case 분석중++++++++++++++
++++++++++++ 829 번째 case 분석중++++++++++++++
++++++++++++ 830 번째 case 분석중++++++++++++++
++++++++++++ 831 번째 case 분석중++++++++++++++
++++++++++++ 832 번째 case 분석중++++++++++++++
++++++++++++ 833 번째 case 분석중++++++++++++++
++++++++++++ 834 번째 case 분석중++++++++++++++
++++++++++++ 835 번째 case 분석중++++++++++++++
++++++++++++ 836 번째 case 분석중++++++++++++++
++++++++++++ 837 번째 case 분석중++++++++++++++
++++++++++++ 838 번째 case 분석중++++++++++++++
++++++++++++ 839 번째 case 분석중++++++++++++++
++++++++++++ 840 번째 case 분석중++++++++++++++
++++++++++++ 841 번째 case 분석중++++++++++++++
++++++++++++ 842 번째 case 분석중++++++++++++++
++++++++++++ 843 번째 case 분석중++++++++++++++
++++++++++++ 844 번째 case 분석중++++++++++++++
++++++++++++ 845 번째 case 분석중++++++++++++++
++++++++++++ 846 번째 case 분석중++++++++++++++
+++++++++++

++++++++++++ 1015 번째 case 분석중++++++++++++++
++++++++++++ 1016 번째 case 분석중++++++++++++++
++++++++++++ 1017 번째 case 분석중++++++++++++++
++++++++++++ 1018 번째 case 분석중++++++++++++++
++++++++++++ 1019 번째 case 분석중++++++++++++++
++++++++++++ 1020 번째 case 분석중++++++++++++++
++++++++++++ 1021 번째 case 분석중++++++++++++++
++++++++++++ 1022 번째 case 분석중++++++++++++++
++++++++++++ 1023 번째 case 분석중++++++++++++++
++++++++++++ 1024 번째 case 분석중++++++++++++++
++++++++++++ 1025 번째 case 분석중++++++++++++++
++++++++++++ 1026 번째 case 분석중++++++++++++++
++++++++++++ 1027 번째 case 분석중++++++++++++++
++++++++++++ 1028 번째 case 분석중++++++++++++++
++++++++++++ 1029 번째 case 분석중++++++++++++++
++++++++++++ 1030 번째 case 분석중++++++++++++++
++++++++++++ 1031 번째 case 분석중++++++++++++++
++++++++++++ 1032 번째 case 분석중++++++++++++++
++++++++++++ 1033 번째 case 분석중++++++++++++++
++++++++++++ 1034 번째 case 분석중++++++++++++++
++++++++++++ 1035 번째 case 분석중++++++++++++++
++++++++++++ 1036 번째 case 분석중++++++++++++++
++++++++++++ 1037 번째 case 분석중+++

# 3. Creating New Variables Related to Lyrics

### 1) num_words

In [165]:
emo_df.head()

,id,title,label,token_lyric_twitter,positive,negative,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,5398990,이 소설의 끝을 다시 써보려 해,1,"[시계, 가, 반대, 로, 돌아가고, 있어, TV, 속, 영화, 가, 되, 감아, ...",0.045455,21.0,9.0,1.0,18.0,2.0,1.0,6.0,0.0,5.0
1,9634954,러시안 룰렛 (Russian Roulette),0,"[La, La, La, La, La, La, La, La, La, La, La, L...",0.312500,11.0,2.0,9.0,12.0,0.0,4.0,2.0,1.0,1.0
2,9614761,그대라는 사치,0,"[그림, 같은, 집, 이, 뭐, 별거, 겠어, 요, 어느, 곳, 이든, 그대, 가,...",0.545455,15.0,3.0,11.0,11.0,0.0,15.0,1.0,0.0,7.0
3,9647324,Dancing King,0,"[Hey, hey, hey, heyHey, hey, hey, heyHey, hey,...",0.000000,19.0,4.0,0.0,18.0,1.0,0.0,2.0,0.0,1.0
4,9613173,목요일 밤 (Feat. 빈지노),0,"[평범한, 목요일, 밤널, 데려, 갈, 게, 어디, 든, 일주일, 중, 에, 네, ...",0.384615,8.0,3.0,7.0,8.0,1.0,3.0,5.0,1.0,5.0


In [185]:
# count the numbr of tokens
emo_df['num_words'] = 0

for i in range(0, 1065):
    emo_df['num_words'].iloc[i] = len(emo_df['token_lyric_twitter'].iloc[i])

/Users/jeongseohyeong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### 2) pos_twitter and pos_count

In [ ]:
emo_df['pos_twitter'] = df_small['lyric'].apply(tokenizer_twitter_pos)

def pos_counter(lst):
    d = {}
    for x, y in lst:
        d[y] = d.get(y, 0) + 1
    return d

emo_df['pos_count'] = emo_df['pos_twitter'].apply(pos_counter)

In [208]:
emo_df.head()

,id,title,label,token_lyric_twitter,positive,negative,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,num_words,pos_twitter,pos_count
0,5398990,이 소설의 끝을 다시 써보려 해,1,"[시계, 가, 반대, 로, 돌아가고, 있어, TV, 속, 영화, 가, 되, 감아, ...",1.0,21.0,9.0,1.0,18.0,2.0,1.0,6.0,0.0,5.0,247.0,"[(시계, Noun), (가, Josa), (반대, Noun), (로, Josa),...","{'Noun': 102, 'Josa': 53, 'Verb': 39, 'Adjecti..."
1,9634954,러시안 룰렛 (Russian Roulette),0,"[La, La, La, La, La, La, La, La, La, La, La, L...",5.0,11.0,2.0,9.0,12.0,0.0,4.0,2.0,1.0,1.0,400.0,"[(La, Alpha), (La, Alpha), (La, Alpha), (La, A...","{'Alpha': 163, 'Adjective': 14, 'Verb': 52, 'N..."
2,9614761,그대라는 사치,0,"[그림, 같은, 집, 이, 뭐, 별거, 겠어, 요, 어느, 곳, 이든, 그대, 가,...",18.0,15.0,3.0,11.0,11.0,0.0,15.0,1.0,0.0,7.0,299.0,"[(그림, Noun), (같다, Adjective), (집, Noun), (이, J...","{'Noun': 141, 'Adjective': 25, 'Josa': 53, 'Ve..."
3,9647324,Dancing King,0,"[Hey, hey, hey, heyHey, hey, hey, heyHey, hey,...",0.0,19.0,4.0,0.0,18.0,1.0,0.0,2.0,0.0,1.0,393.0,"[(Hey, Alpha), (hey, Alpha), (hey, Alpha), (he...","{'Alpha': 111, 'Adjective': 13, 'Noun': 116, '..."
4,9613173,목요일 밤 (Feat. 빈지노),0,"[평범한, 목요일, 밤널, 데려, 갈, 게, 어디, 든, 일주일, 중, 에, 네, ...",5.0,8.0,3.0,7.0,8.0,1.0,3.0,5.0,1.0,5.0,392.0,"[(평범하다, Adjective), (목요일, Noun), (밤널, Noun), (...","{'Adjective': 19, 'Noun': 135, 'Verb': 44, 'Jo..."


### 3) n_lex: number of nouns, adjectives, verbs, adverbs

In [222]:
emo_df['N_lex'] = 0


for i in range(0, len(emo_df)):
    
    dic = emo_df['pos_count'].iloc[i]
    
    if 'Noun' in dic:
        noun = dic.get('Noun')
    else: 
        noun = 0
    if 'Adjective' in dic:
        adjective = dic.get('Adjective')
    else:
        adjective = 0
    if 'Verb' in dic:
        verb = dic.get('Verb')
    else:
        verb = 0
    if 'Adverb' in dic:
        adverb = dic.get('Adverb')
        
    emo_df['N_lex'].iloc[i] = noun + adjective + verb + adverb

/Users/jeongseohyeong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### 4) lexical_density

ref: https://en.wikipedia.org/wiki/Lexical_density

In [225]:
emo_df['lexical_density'] = 0

for i in range(0, len(emo_df)):
    emo_df['lexical_density'].iloc[i] = emo_df['N_lex'].iloc[i] / len(emo_df['pos_twitter'].iloc[i])
    
emo_df.head()

/Users/jeongseohyeong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,title,label,token_lyric_twitter,positive,negative,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,num_words,pos_twitter,pos_count,N_lex,lecical_density,lexical_density
0,5398990,이 소설의 끝을 다시 써보려 해,1,"[시계, 가, 반대, 로, 돌아가고, 있어, TV, 속, 영화, 가, 되, 감아, ...",1.0,21.0,9.0,1.0,18.0,2.0,1.0,6.0,0.0,5.0,247.0,"[(시계, Noun), (가, Josa), (반대, Noun), (로, Josa),...","{'Noun': 102, 'Josa': 53, 'Verb': 39, 'Adjecti...",162,0,0.726457
1,9634954,러시안 룰렛 (Russian Roulette),0,"[La, La, La, La, La, La, La, La, La, La, La, L...",5.0,11.0,2.0,9.0,12.0,0.0,4.0,2.0,1.0,1.0,400.0,"[(La, Alpha), (La, Alpha), (La, Alpha), (La, A...","{'Alpha': 163, 'Adjective': 14, 'Verb': 52, 'N...",179,0,0.477333
2,9614761,그대라는 사치,0,"[그림, 같은, 집, 이, 뭐, 별거, 겠어, 요, 어느, 곳, 이든, 그대, 가,...",18.0,15.0,3.0,11.0,11.0,0.0,15.0,1.0,0.0,7.0,299.0,"[(그림, Noun), (같다, Adjective), (집, Noun), (이, J...","{'Noun': 141, 'Adjective': 25, 'Josa': 53, 'Ve...",220,0,0.802920
3,9647324,Dancing King,0,"[Hey, hey, hey, heyHey, hey, hey, heyHey, hey,...",0.0,19.0,4.0,0.0,18.0,1.0,0.0,2.0,0.0,1.0,393.0,"[(Hey, Alpha), (hey, Alpha), (hey, Alpha), (he...","{'Alpha': 111, 'Adjective': 13, 'Noun': 116, '...",189,0,0.500000
4,9613173,목요일 밤 (Feat. 빈지노),0,"[평범한, 목요일, 밤널, 데려, 갈, 게, 어디, 든, 일주일, 중, 에, 네, ...",5.0,8.0,3.0,7.0,8.0,1.0,3.0,5.0,1.0,5.0,392.0,"[(평범하다, Adjective), (목요일, Noun), (밤널, Noun), (...","{'Adjective': 19, 'Noun': 135, 'Verb': 44, 'Jo...",202,0,0.547425


In [260]:
# emo_df.to_csv("emo_df_sh.csv") : to save